# 04. Bigram News Group(N-gram)

## 1. 20 Newsgroups

### 1) 20 Newsgroups data 준비

<http://scikit-learn.org/0.19/datasets/twenty_newsgroups.html>

* The 20 newsgroups dataset comprises **around 18000 newsgroups posts on 20 topics** split in two subsets: **one for training** (or development) and **the other one for testing** (or for performance evaluation).
* The split between the train and test set is based upon a **messages posted before and after a specific date.**

---

* train set과 test set이 별도로 분리되어 있음.
* categories 매개변수를 이용하여 20개의 topic 중에서 원하는 topic을 선택할 수 있음
* remove를 이용하여 필요없는 부분을 삭제할 수 있음
* 각 set 내에서 .data 는 text 내용을, .target은 숫자로 변경된 label(category)를 가져오는 데 사용됨
* 따라서 20 newsgroups는 train_test_split을 이용하여 train set과 test set을 분리할 필요가 없음

### 2) category 선정하여 training, test 진행

In [25]:
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
#4개의 카테고리 선택

#학습 및 테스트 진행
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      #메일 내용에서 hint가 되는 부분을 삭제 - 순수하게 내용만으로
                                      categories=categories)   #4개의 카테고리로만 진행

newsgroups_test = fetch_20newsgroups(subset='test', 
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories)

In [26]:
print('train set size:', len(newsgroups_train.data))   #.data로 text 불러오기
print('test set size:', len(newsgroups_test.data))
print('selected categories:', newsgroups_train.target_names)   #.target은 카테고리 숫자, _names는 이름 불러오기
print('train labels:', set(newsgroups_train.target))   #카테고리 label 확인하기

train set size: 2034
test set size: 1353
selected categories: ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']
train labels: {0, 1, 2, 3}


In [27]:
# train, test set text 및 label 결과 확인하기
print('##Train set text samples:', newsgroups_train.data[0])
print('##Train set label smaples:', newsgroups_train.target[0])
print('##Test set text samples:', newsgroups_test.data[0])
print('##Test set label smaples:', newsgroups_test.target[0])

##Train set text samples: Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych
##Train set label smaples: 1
##Test set text samples: TRry the SKywatch project in  Arizona.
##Test set label smaples: 2


### 3) text document classification
* newsgroups_train과 newsgroups_test로부터 .data와 .target을 이용하여 X_train, X_test, y_train, y_test을 추출하여 text document classification을 수행하시오.

In [28]:
# text 추출
X_train = newsgroups_train.data
X_test = newsgroups_test.data

# label(카테고리) 추출
y_train = newsgroups_train.target
y_test = newsgroups_test.target

In [29]:
# stopwords(자주 쓰이지만 분석에 의미가 없는 단어들) 중 영어만 추출
from nltk.corpus import stopwords
cachedStopWords = stopwords.words("english")

In [30]:
# text로부터 TFIDF 직접생성
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(token_pattern= "[a-zA-Z']{3,}",   # 알파벳 3개 이상의 단어만 tokenize 
                        decode_error ='ignore',    #decode error 무시
                        lowercase=True,    #소문자로 변경
                        stop_words = stopwords.words('english'), 
                        max_df=0.5,   #단어가 포함된 최대 문서 비율(문서 전체 중 50%가 넘게 들어가 있는 단어는 제외)
                        min_df=2    #단어가 포함된 최소 문서 개수
                       ).fit(X_train)

X_train_tfidf = tfidf.transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

print(X_train_tfidf.shape)

(2034, 11483)


In [31]:
# 학습 및 성능체크
from sklearn.linear_model import LogisticRegression 
clf = LogisticRegression() #분류기 선언
clf.fit(X_train_tfidf, y_train) # train data를 이용하여 분류기를 학습
print('Train set score: {:.3f}'.format(clf.score(X_train_tfidf, y_train))) # train data에 대한 예측정확도 
print('Test set score: {:.3f}'.format(clf.score(X_test_tfidf, y_test))) # test data에 대한 예측정확도

Train set score: 0.966
Test set score: 0.761


## 2. N-gram
* 문맥(context)를 파악하기 위한 전통적 방법
* 대상이 되는 문자열을 하나의 단어 단위가 아닌, 두 개 이상의 단위로 잘라서 처리
* 보통 unigram에 bi-gram, tri-gram을 추가하면서 feature의 수가 증가시켜 사용

### 1) Bigram
* text, 즉 문자열을 2개의 단어씩 잘라서 tokenize하는 방법

In [32]:
tfidf = TfidfVectorizer(token_pattern= "[a-zA-Z']{3,}", 
                        decode_error ='ignore', 
                        lowercase=True, 
                        stop_words = stopwords.words('english'),
                        ngram_range=(1, 2),   #bigram으로 설정
                        max_df=0.5,
                        min_df=2).fit(X_train)
X_train_tfidf = tfidf.transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

print(X_train_tfidf.shape)

(2034, 26550)


In [33]:
bigram_features = [f for f in tfidf.get_feature_names() if len(f.split()) > 1]
print(bigram_features[:10])

["'cause can't", "'em better", "'expected errors'", "'karla' next", "'nodis' password", "'official doctrine", "'ok see", "'sci astro'", "'what's moonbase", 'aas american']


In [34]:
clf = LogisticRegression() #분류기 선언
clf.fit(X_train_tfidf, y_train) # train data를 이용하여 분류기를 학습
print('Train set score: {:.3f}'.format(clf.score(X_train_tfidf, y_train))) # train data에 대한 예측정확도 
print('Test set score: {:.3f}'.format(clf.score(X_test_tfidf, y_test))) # test data에 대한 예측정확도

# train score는 unigram 보다 상승, test score는 성능이 떨어짐

Train set score: 0.969
Test set score: 0.756


### 2) Trigram
* 문자열을 3개의 단어씩 끊어서 tokenize

In [35]:
tfidf = TfidfVectorizer(token_pattern= "[a-zA-Z']{3,}", 
                        decode_error ='ignore', 
                        lowercase=True, 
                        stop_words = stopwords.words('english'),
                        ngram_range=(1, 3),  #trigram
                        max_df=0.5,
                        min_df=2).fit(X_train)
X_train_tfidf = tfidf.transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

print(X_train_tfidf.shape)

(2034, 32943)


In [36]:
trigram_features = [f for f in tfidf.get_feature_names() if len(f.split()) > 2]
print(trigram_features[:10])

["'em better shots", "'expected errors' basically", "'karla' next one", "'nodis' password also", "'official doctrine think", "'ok see warning", "'what's moonbase good", 'aas american astronautical', 'ability means infallible', 'able accept donations']


In [37]:
clf = LogisticRegression() #분류기 선언
clf.fit(X_train_tfidf, y_train) # train data를 이용하여 분류기를 학습
print('Train set score: {:.3f}'.format(clf.score(X_train_tfidf, y_train))) # train data에 대한 예측정확도 
print('Test set score: {:.3f}'.format(clf.score(X_test_tfidf, y_test))) # test data에 대한 예측정확도

# train score는 unigram보다 상승, test는 성능이 떨어짐

Train set score: 0.969
Test set score: 0.758


### 3) Ridge

In [38]:
from sklearn.linear_model import RidgeClassifier
ridge_clf = RidgeClassifier() #릿지 분류기 선언
ridge_clf.fit(X_train_tfidf, y_train) #학습
print('Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))

# train, test score 모두 현재까지 가장 좋은 성능

Train set score: 0.976
Test set score: 0.775


### 4) Lasso

In [39]:
import numpy as np
lasso_clf = LogisticRegression(penalty='l1', solver='liblinear') # Lasso는 동일한 LogisticRegression을 사용하면서 매개변수로 지정
lasso_clf.fit(X_train_tfidf, y_train) # train data로 학습
print('Train set score: {:.3f}'.format(lasso_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(lasso_clf.score(X_test_tfidf, y_test)))
print('Used features count: {}'.format(np.sum(lasso_clf.coef_ != 0)), 'out of', X_train_tfidf.shape[1]) 

# 현재까지 train, test 모두 성능이 가장 떨어짐

Train set score: 0.761
Test set score: 0.695
Used features count: 246 out of 32943


### 5) SVM(Support Vector Machine)
* 결정 경계(Decision Boundary), 즉 분류를 위한 기준 선을 정의하는 모델
* 분류되지 않은 새로운 점이 나타나면 경계의 어느 쪽에 속하는 지 확인해서 분류 과제를 수행

* 데이터 속성(feature)가 2개 일 때, 결정 경계 그래프 예시
![alt text](https://i0.wp.com/hleecaster.com/wp-content/uploads/2020/01/svm01.png?w=1372 "wp-image-5017 jetpack-lazy-image jetpack-lazy-image--handled")

In [40]:
from sklearn.svm import SVC
clf = SVC(gamma='auto', kernel='linear')
clf.fit(X_train_tfidf, y_train) 
print('Train set score: {:.3f}'.format(clf.score(X_train_tfidf, y_train))) # train data에 대한 예측정확도 
print('Test set score: {:.3f}'.format(clf.score(X_test_tfidf, y_test))) # test data에 대한 예측정확도

# train score은 ridge 다음으로 높고, test는 ridge, unigram 다음으로 높음(3 번째)

Train set score: 0.974
Test set score: 0.758


#### train, test set score 비교하기

In [42]:
import pandas as pd
scores = pd.read_csv('C:/Users/Okyoung/Desktop/Business Analytics_캡스톤디자인/BusinessAnalytics/HW/compare_score2.csv')

scores   #ridge가 train, test 모두 가장 좋은 성능을 보유

,method,train,test
0,uni-gram,0.966,0.761
1,bi-gram,0.969,0.756
2,tri-gram,0.969,0.758
3,ridge,0.976,0.775
4,lasso,0.761,0.695
5,svm,0.974,0.758
